In [1]:
import scanpy as sc
import numpy as np

In [2]:
adata = sc.read_h5ad("/cristealab/xiwang/CellXGene/normal_after_20230515/merged_h5ad/merged_normal.h5ad")

/cristealab/xiwang/miniconda3/envs/llm_sc_benchmark/lib/python3.10/site-packages/anndata/_core/anndata.py:1756: UserWarning: Observation names are not unique. To make them unique, call `.obs_names_make_unique`.
  utils.warn_names_duplicates("obs")


In [3]:
adata

AnnData object with n_obs × n_vars = 7974542 × 60530
    obs: 'soma_joinid', 'dataset_id', 'assay', 'assay_ontology_term_id', 'cell_type', 'cell_type_ontology_term_id', 'development_stage', 'development_stage_ontology_term_id', 'disease', 'disease_ontology_term_id', 'donor_id', 'is_primary_data', 'observation_joinid', 'self_reported_ethnicity', 'self_reported_ethnicity_ontology_term_id', 'sex', 'sex_ontology_term_id', 'suspension_type', 'tissue', 'tissue_ontology_term_id', 'tissue_type', 'tissue_general', 'tissue_general_ontology_term_id', 'raw_sum', 'nnz', 'raw_mean_nnz', 'raw_variance_nnz', 'n_measured_vars'
    var: 'soma_joinid', 'feature_id', 'feature_name', 'feature_length', 'nnz', 'n_measured_obs'

In [4]:
# -------------------------------------------------------------------
# 2. Define the (dataset_id, tissue) pairs you want to keep
# -------------------------------------------------------------------
desired_pairs = [
    ("d6505c89-c43d-4c28-8c4f-7351a5fd5528", "peripheral region of retina"),
    ("1a38e762-2465-418f-b81c-6a4bce261c34", "cerebral cortex"),
    ("2c820d53-cbd7-4c28-8c4f-7351a5fd5528", "blood"),
    ("55003f67-c494-46f1-83fb-902745646379", "breast"),
    ("093d3bfe-6f0f-4ac0-a7a1-829f94d0a49f", "lung"),
    ("093d3bfe-6f0f-4ac0-a7a1-829f94d0a49f", "trachea"),
    ("bab7432a-5cfe-45ea-928c-422d03c45cdd", "cerebellum"),
    ("5af90777-6760-4003-9dba-8f945fec6fdf", "kidney"),
    ("965386e9-1e4f-466d-bf59-ebdca4b66b9b", "bone marrow"),
]

# -------------------------------------------------------------------
# 3. Subset to these pairs (ALL cells)
# -------------------------------------------------------------------
# Create a helper column that combines dataset_id and tissue into a tuple
adata.obs["pair"] = list(zip(adata.obs["dataset_id"], adata.obs["tissue"]))

# Convert desired pairs to a set for faster membership checks
desired_pairs_set = set(desired_pairs)

# Build a boolean mask for the desired pairs
mask = adata.obs["pair"].isin(desired_pairs_set)
adata_subset_all = adata[mask].copy()
print(f"Subset (all cells) AnnData: {adata_subset_all}")

# Drop the helper column if you like
adata_subset_all.obs.drop(columns=["pair"], inplace=True)

# Save the subset with all cells
adata_subset_all.write_h5ad("/cristealab/xiwang/Outputs/llm_sc_benchmark/cellxgene_val_samples_datasets/h5ad/subset_all_cells.h5ad")
print("Saved '/cristealab/xiwang/Outputs/llm_sc_benchmark/cellxgene_val_samples_datasets/h5ad/subset_all_cells.h5ad'")


/cristealab/xiwang/miniconda3/envs/llm_sc_benchmark/lib/python3.10/site-packages/anndata/_core/anndata.py:1756: UserWarning: Observation names are not unique. To make them unique, call `.obs_names_make_unique`.
  utils.warn_names_duplicates("obs")


Subset (all cells) AnnData: AnnData object with n_obs × n_vars = 1901333 × 60530
    obs: 'soma_joinid', 'dataset_id', 'assay', 'assay_ontology_term_id', 'cell_type', 'cell_type_ontology_term_id', 'development_stage', 'development_stage_ontology_term_id', 'disease', 'disease_ontology_term_id', 'donor_id', 'is_primary_data', 'observation_joinid', 'self_reported_ethnicity', 'self_reported_ethnicity_ontology_term_id', 'sex', 'sex_ontology_term_id', 'suspension_type', 'tissue', 'tissue_ontology_term_id', 'tissue_type', 'tissue_general', 'tissue_general_ontology_term_id', 'raw_sum', 'nnz', 'raw_mean_nnz', 'raw_variance_nnz', 'n_measured_vars', 'pair'
    var: 'soma_joinid', 'feature_id', 'feature_name', 'feature_length', 'nnz', 'n_measured_obs'
Saved '/cristealab/xiwang/Outputs/llm_sc_benchmark/cellxgene_val_samples_datasets/h5ad/subset_all_cells.h5ad'


In [4]:
adata_subset_all = sc.read_h5ad("/cristealab/xiwang/Outputs/llm_sc_benchmark/cellxgene_val_samples_datasets/subset_all_cells/h5ad/subset_all_cells.h5ad")

/cristealab/xiwang/miniconda3/envs/llm_sc_benchmark/lib/python3.10/site-packages/anndata/_core/anndata.py:1756: UserWarning: Observation names are not unique. To make them unique, call `.obs_names_make_unique`.
  utils.warn_names_duplicates("obs")


In [5]:
desired_pairs = [
    ("d6505c89-c43d-4c28-8c4f-7351a5fd5528", "peripheral region of retina"),
    ("1a38e762-2465-418f-b81c-6a4bce261c34", "cerebral cortex"),
    ("2c820d53-cbd7-4c28-8c4f-7351a5fd5528", "blood"),
    ("55003f67-c494-46f1-83fb-902745646379", "breast"),
    ("093d3bfe-6f0f-4ac0-a7a1-829f94d0a49f", "lung"),
    ("093d3bfe-6f0f-4ac0-a7a1-829f94d0a49f", "trachea"),
    ("bab7432a-5cfe-45ea-928c-422d03c45cdd", "cerebellum"),
    ("5af90777-6760-4003-9dba-8f945fec6fdf", "kidney"),
    ("965386e9-1e4f-466d-bf59-ebdca4b66b9b", "bone marrow"),
]

In [6]:
# -------------------------------------------------------------------
# 4. Randomly sample up to 1,000 cells from each pair (recreating 'pair' if dropped)
# -------------------------------------------------------------------
# If you dropped 'pair' above, uncomment:
# adata_subset_all.obs["pair"] = list(zip(adata_subset_all.obs["dataset_id"], 
#                                         adata_subset_all.obs["tissue"]))

rng = np.random.default_rng(seed=42)  # For reproducibility
sampled_list = []

for dset_id, tissue_type in desired_pairs:
    # Filter for cells belonging to this (dataset_id, tissue)
    pair_mask = (
        (adata_subset_all.obs["dataset_id"] == dset_id) &
        (adata_subset_all.obs["tissue"] == tissue_type)
    )
    adata_pair = adata_subset_all[pair_mask]

    n_obs = adata_pair.n_obs
    if n_obs == 0:
        print(f"No cells found for ({dset_id}, {tissue_type}) — skipping.")
        continue

    # We want up to 1,000 cells
    n_take = min(1250, n_obs)
    selected_indices = rng.choice(n_obs, size=n_take, replace=False)
    
    # Create the sampled subset
    adata_sampled = adata_pair[selected_indices, :].copy()
    sampled_list.append(adata_sampled)
    print(f"({dset_id}, {tissue_type}): took {n_take} cells (of {n_obs}).")

# -------------------------------------------------------------------
# 5. Concatenate all sampled subsets and save
# -------------------------------------------------------------------
if len(sampled_list) == 0:
    raise ValueError("No data was sampled. Please check your pairs or source data.")

# Use scanpy's concatenate. Each subset is assigned a 'batch' category.
batch_categories = [f"{p[0]}_{p[1]}" for p in desired_pairs]
adata_subset_1k = sampled_list[0].concatenate(
    *sampled_list[1:], 
    join="outer", 
    batch_key="sample_source",
    batch_categories=batch_categories
)

# Drop the helper column if present
if "pair" in adata_subset_1k.obs:
    adata_subset_1k.obs.drop(columns=["pair"], inplace=True)

print(f"Final sampled AnnData: {adata_subset_1k}")
adata_subset_1k.write_h5ad("/cristealab/xiwang/DSR1_Preprint/Outputs/cellxgene_cutoff_8datasets_10k/h5ad/subset_10k_cells.h5ad")
print("Saved '/cristealab/xiwang/DSR1_Preprint/Outputs/cellxgene_cutoff_8datasets_10k/h5ad/subset_10k_cells.h5ad'")

(d6505c89-c43d-4c28-8c4f-7351a5fd5528, peripheral region of retina): took 1250 cells (of 1062127).
(1a38e762-2465-418f-b81c-6a4bce261c34, cerebral cortex): took 1250 cells (of 39256).
No cells found for (2c820d53-cbd7-4c28-8c4f-7351a5fd5528, blood) — skipping.
(55003f67-c494-46f1-83fb-902745646379, breast): took 1250 cells (of 551770).
(093d3bfe-6f0f-4ac0-a7a1-829f94d0a49f, lung): took 1250 cells (of 46325).
(093d3bfe-6f0f-4ac0-a7a1-829f94d0a49f, trachea): took 1250 cells (of 28392).
(bab7432a-5cfe-45ea-928c-422d03c45cdd, cerebellum): took 1250 cells (of 128695).
(5af90777-6760-4003-9dba-8f945fec6fdf, kidney): took 1250 cells (of 11838).
(965386e9-1e4f-466d-bf59-ebdca4b66b9b, bone marrow): took 1250 cells (of 32930).


/tmp/ipykernel_186107/2294434225.py:41: FutureWarning: Use anndata.concat instead of AnnData.concatenate, AnnData.concatenate is deprecated and will be removed in the future. See the tutorial for concat at: https://anndata.readthedocs.io/en/latest/concatenation.html
  adata_subset_1k = sampled_list[0].concatenate(
/cristealab/xiwang/miniconda3/envs/llm_sc_benchmark/lib/python3.10/site-packages/anndata/_core/anndata.py:1756: UserWarning: Observation names are not unique. To make them unique, call `.obs_names_make_unique`.
  utils.warn_names_duplicates("obs")
/cristealab/xiwang/miniconda3/envs/llm_sc_benchmark/lib/python3.10/site-packages/anndata/_core/anndata.py:1756: UserWarning: Observation names are not unique. To make them unique, call `.obs_names_make_unique`.
  utils.warn_names_duplicates("obs")
/cristealab/xiwang/miniconda3/envs/llm_sc_benchmark/lib/python3.10/site-packages/anndata/_core/anndata.py:1756: UserWarning: Observation names are not unique. To make them unique, call `.o

Final sampled AnnData: AnnData object with n_obs × n_vars = 10000 × 60530
    obs: 'soma_joinid', 'dataset_id', 'assay', 'assay_ontology_term_id', 'cell_type', 'cell_type_ontology_term_id', 'development_stage', 'development_stage_ontology_term_id', 'disease', 'disease_ontology_term_id', 'donor_id', 'is_primary_data', 'observation_joinid', 'self_reported_ethnicity', 'self_reported_ethnicity_ontology_term_id', 'sex', 'sex_ontology_term_id', 'suspension_type', 'tissue', 'tissue_ontology_term_id', 'tissue_type', 'tissue_general', 'tissue_general_ontology_term_id', 'raw_sum', 'nnz', 'raw_mean_nnz', 'raw_variance_nnz', 'n_measured_vars', 'sample_source'
    var: 'soma_joinid', 'feature_id', 'feature_name', 'feature_length', 'nnz', 'n_measured_obs'
Saved '/cristealab/xiwang/DSR1_Preprint/Outputs/cellxgene_cutoff_8datasets_10k/h5ad/subset_10k_cells.h5ad'


In [6]:
adata_subset_1k

AnnData object with n_obs × n_vars = 8000 × 60530
    obs: 'soma_joinid', 'dataset_id', 'assay', 'assay_ontology_term_id', 'cell_type', 'cell_type_ontology_term_id', 'development_stage', 'development_stage_ontology_term_id', 'disease', 'disease_ontology_term_id', 'donor_id', 'is_primary_data', 'observation_joinid', 'self_reported_ethnicity', 'self_reported_ethnicity_ontology_term_id', 'sex', 'sex_ontology_term_id', 'suspension_type', 'tissue', 'tissue_ontology_term_id', 'tissue_type', 'tissue_general', 'tissue_general_ontology_term_id', 'raw_sum', 'nnz', 'raw_mean_nnz', 'raw_variance_nnz', 'n_measured_vars', 'sample_source'
    var: 'soma_joinid', 'feature_id', 'feature_name', 'feature_length', 'nnz', 'n_measured_obs'